In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('./Online Retail.csv')


In [31]:

null_counts = df.isnull().sum()
null_counts


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
rate                0
dtype: int64

In [32]:
num_unique_values = df['InvoiceNo'].nunique()
num_unique_values 


25900

In [3]:

df.shape[0]


541909

In [34]:
# Lọc bỏ các dòng có 'InvoiceNo' bắt đầu bằng chữ 'C', 
# do các đơnInvoiceNo: Số hóa đơn. Nominal, một số nguyên gồm 6 chữ số được gán duy nhất cho mỗi giao dịch.
# Nếu mã này bắt đầu bằng chữ cái 'c', thì nó biểu thị một lệnh hủy.
df = df[~df['InvoiceNo'].str.startswith('C')]
df.shape[0]


532621

In [35]:
# Loại bỏ các dòng có giá trị NaN trong cột 'price
df = df.dropna(subset=['CustomerID'])
df.shape[0]

397924

In [36]:
# Giữ lại những hàng có `StockCode` chứa ít nhất một chữ số
df = df[df['StockCode'].str.contains(r'\d')]
df.shape[0]


396503

In [37]:
def transform_stock_code(stock_code):
    # Tách phần số và phần chữ cái
    base_code = ''.join(filter(str.isdigit, stock_code))  # Lấy phần số
    suffix = ''.join(filter(str.isalpha, stock_code))     # Lấy phần chữ

    # Kiểm tra nếu có phần chữ cái (sẽ thêm 10 + thứ tự của chữ cái)
    if suffix:
        suffix_value = ord(suffix[0].upper()) - ord('A') + 1  # Đổi chữ cái đầu tiên thành số
        return int(base_code) * 100 + 10 + suffix_value
    else:
        return int(base_code)

# Tạo cột mới 'stockCodeTransform'
df['stockCodeTransform'] = df['StockCode'].apply(transform_stock_code)

# Kiểm tra kết quả
print(df[['StockCode', 'stockCodeTransform']].head())


  StockCode  stockCodeTransform
0    85123A             8512311
1     71053               71053
2    84406B             8440612
3    84029G             8402917
4    84029E             8402915


In [38]:
null_counts = df.isnull().sum()
null_counts

InvoiceNo             0
StockCode             0
Description           0
Quantity              0
InvoiceDate           0
UnitPrice             0
CustomerID            0
Country               0
rate                  0
stockCodeTransform    0
dtype: int64

In [39]:
df.shape[0]

396503

In [40]:
# Step 1: Add a 'Utility' column to the dataset and convert to integer
df['Utility'] = (df['UnitPrice'] * df['Quantity'] * 100).astype(int)

# Step 2: Define preprocessing functions
def filter_large_transactions(df, item_count_threshold=100):
    """Remove transactions with more than the specified number of items."""
    item_counts = df.groupby('InvoiceNo').size()
    valid_invoices = item_counts[item_counts <= item_count_threshold].index
    return df[df['InvoiceNo'].isin(valid_invoices)]

def filter_high_utility_items(df, utility_threshold=1000):
    """Remove items with utility greater than the threshold times the average utility."""
    avg_utility = df['Utility'].mean()
    max_utility = avg_utility * utility_threshold
    return df[df['Utility'] <= max_utility]

# Step 3: Apply filters
# Filter transactions with more than 100 items
df = filter_large_transactions(df, item_count_threshold=100)

# Filter items with utility values above the threshold
df = filter_high_utility_items(df, utility_threshold=1000)

In [41]:
df.shape[0]

359939

In [42]:
# Lấy danh sách duy nhất các item dựa trên 'stockCodeTransform'
items = df[['stockCodeTransform', 'StockCode', 'Description', 'UnitPrice', 'Quantity']]

# Tính giá trung bình cho mỗi 'stockCodeTransform' và làm tròn đến 2 chữ số thập phân
items_avg_price = (
    items.groupby(['stockCodeTransform', 'StockCode', 'Description'])
    .agg({'UnitPrice': 'mean', 'Quantity': 'mean'})  # Thêm tính trung bình cho Quantity
    .reset_index()
)

# Làm tròn giá trung bình của UnitPrice đến 2 chữ số thập phân
items_avg_price['UnitPrice'] = items_avg_price['UnitPrice'].round(2)
items_avg_price['Quantity'] = items_avg_price['Quantity'].round().astype(int)  # Làm tròn Quantity về số nguyên

# Giữ lại duy nhất theo 'stockCodeTransform'
# Nếu có nhiều dòng tương ứng cho cùng một 'stockCodeTransform', giữ dòng đầu tiên
items = items_avg_price.drop_duplicates(subset='stockCodeTransform').reset_index(drop=True)

# Thêm một cột 'Index' với thứ tự tăng dần bắt đầu từ 1
items['Index'] = range(1, len(items) + 1)

# Lưu vào tệp CSV
items.to_csv('items.csv', index=False)

print("Đã lưu kết quả vào file 'items.csv'")


Đã lưu kết quả vào file 'items.csv'


In [43]:
# Lấy danh sách duy nhất các itemName và feature
users = df[['CustomerID']].drop_duplicates().reset_index(drop=True)

# Chọn cột cần thiết
users = users[['CustomerID']]

# Sắp xếp theo thứ tự tăng dần của cột 'CustomerID'
users = users.sort_values(by='CustomerID').reset_index(drop=True)

# Thêm một cột 'Index' với thứ tự tăng dần bắt đầu từ 1
users['Index'] = range(1, len(users) + 1)

# Lưu vào tệp CSV sau khi sắp xếp và thêm cột thứ tự
users.to_csv('users.csv', index=False)


In [44]:
# Chọn các cột cần thiết
ratings = df[['CustomerID', 'stockCodeTransform', 'rate']]

# Lưu dữ liệuvào tệp 'ratings.csv'
ratings.to_csv('ratings.csv', index=False)

# Sắp xếp theo thứ tự tăng dần của cột 'CustomerID'
ratings = ratings.sort_values(by='CustomerID').reset_index(drop=True)


# Tạo một dictionary để ánh xạ 'CustomerID' sang 'Index' trong file 'users.csv'
user_mapping = dict(zip(users['CustomerID'], users['Index']))

# Tạo một dictionary để ánh xạ 'StockCode' sang 'Index' trong file 'items.csv'
item_mapping = dict(zip(items['stockCodeTransform'], items['Index']))

# Thay thế 'CustomerID' bằng 'Index' trong ratings
ratings['CustomerID'] = ratings['CustomerID'].map(user_mapping)

# Thay thế 'stockCodeTransform' bằng 'Index' trong ratings
ratings['stockCodeTransform'] = ratings['stockCodeTransform'].map(item_mapping)

# Lưu dữ liệu vào tệp 'ratings.csv' sau khi thay thế
ratings.to_csv('ratings.csv', index=False)



In [45]:
# Lưu vào tệp CSV
df.to_csv('final_clean.csv', index=False)

In [46]:
df.shape[0]

359939

In [47]:
num_unique_values = df['InvoiceNo'].nunique()
num_unique_values 

18160

In [48]:
df.shape[0]

359939

In [49]:
# Nhóm dữ liệu theo Invoice, tạo danh sách các StockCode, tổng utility và danh sách utility cho mỗi Invoice
grouped_data = df.groupby('InvoiceNo').apply(
    lambda x: {
        'itemids': ' '.join(map(str, x['stockCodeTransform'])),
        'total_utility': int(x['Utility'].sum()),
        'utilities': ' '.join(map(str, x['Utility'].astype(int)))
    }
).apply(pd.Series)

# Định dạng đầu ra theo yêu cầu
formatted_data = grouped_data.apply(
    lambda row: f"{row['itemids']}:{row['total_utility']}:{row['utilities']}", axis=1
)

# Lưu kết quả vào file 'final_transactions.txt'
output_path = 'final_transactions.txt'
formatted_data.to_csv(output_path, index=False, header=False)

output_path


'final_transactions.txt'

In [50]:
num_unique_values = df['InvoiceNo'].nunique()
num_unique_values 

18160